## Load Packages

In [1]:
# # Install tempo-embeddings from GitHub
# # This can also refer to a specific version or branch

%pip install -e ..

# %pip install --upgrade pip  # Required for properly resolving dependencies
# %pip uninstall -y tempo_embeddings  # Remove existing installation
# %pip install --upgrade git+https://github.com/Semantics-of-Sustainability/tempo-embeddings.git

Obtaining file:///Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for tempo_embeddings (pyproject.toml) ... done
  Created wheel for tempo_embeddings: filename=tempo_embeddings-0.0.1-0.editable-py3-none-any.whl size=9598 sha256=3a0c057fa7491bc70a3e36a984d79c8af09593695a0e97c7973f60375bf53d99
  Stored in directory: /private/var/folders/d8/j5_fyf8941j_492zvf8948y40000gn/T/pip-ephem-wheel-cache-n6bfowec/wheels/9d/dd/1c/d0e8ab6bde11a84556c00bd645333aaad0b90e2c38e11b0ed1
Successfully built tempo_embeddings
  Attempting uninstall: tempo_embeddings
    Found existing installation: tempo_embeddings 0.0.1
    Uninstalling tempo_embeddings-0.0.1:
      Successfully uninstalled tempo_embeddings-0.0.1
Note: you may need to restart the ke

In [2]:
# make sure installation has succeeded

import logging
from importlib import reload

reload(logging)
logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)

In [3]:
%load_ext autoreload
%autoreload now

In [4]:
try:
    import google.colab  # noqa: F401

    IN_COLAB = True
except ModuleNotFoundError:
    IN_COLAB = False

In [5]:
from pathlib import Path

from tempo_embeddings.text.corpus import Corpus

Using corpus directory: '/Volumes/i-lab.data.uu.nl/research-semantics-of-sustainability/semantics-of-sustainability/data'


## Load Database Manager

The `db_path` parameter should point to the directory where the database is, so the original configuration and records are loaded. The database was created using the notebook `1_compute_embeddings_nl.ipynb`. If the given path does not exist, a new EMPTY database will be created there. 

A bigger `batch_size` could make the search faster but if it is too big you might run out of memory.

In [6]:
import weaviate
from tempo_embeddings.embeddings.model import SentenceTransformerModelWrapper
from tempo_embeddings.embeddings.weaviate_database import WeaviateDatabaseManager
from tempo_embeddings.settings import DEFAULT_LANGUAGE_MODEL

db = WeaviateDatabaseManager(
    client=weaviate.connect_to_local(host="145.38.192.173", port=8087),
    model=SentenceTransformerModelWrapper.from_pretrained(DEFAULT_LANGUAGE_MODEL),
)

/Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings/.venv/lib/python3.11/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(
04:44:55 INFO:HTTP Request: GET http://localhost:8087/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
04:44:55 INFO:HTTP Request: GET http://localhost:8087/v1/meta "HTTP/1.1 200 OK"
04:44:55 INFO:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
/Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
04:44:56 INFO:HTTP Request: GET http://localhost:8087/v1/schema/TempoEmbeddings "HTTP/1.1 200 OK"


## Choose from the available Collections in the Database

In [7]:
from ipywidgets import widgets

existing_colls = db.get_available_collections()
collection_selector = widgets.SelectMultiple(
    options=existing_colls,
    value=["ANP", "StatenGeneraal"],
    description="Choose a Collection:",
    disabled=False,
    style={"description_width": "initial"},
)
collection_selector

04:44:56 INFO:HTTP Request: POST http://localhost:8087/v1/graphql "HTTP/1.1 200 OK"


SelectMultiple(description='Choose a Collection:', index=(2, 0), options=('StatenGeneraal', 'AlgemeenDagblad',…

## Show number of records in the selected collection

In [8]:
for collection in collection_selector.value:
    print(
        f"Collection '{collection}' has {db.get_collection_count(collection)} records"
    )

04:44:56 INFO:HTTP Request: POST http://localhost:8087/v1/graphql "HTTP/1.1 200 OK"
04:44:56 INFO:HTTP Request: POST http://localhost:8087/v1/graphql "HTTP/1.1 200 OK"


Collection 'ANP' has 126182 records
Collection 'StatenGeneraal' has 794594 records


## Create Sub-Corpus

To make the processing and visualization easier, we will create a new `Corpus` comprising only a subet of the original Collection. This corpus will contain only the records of interest. This is done by querying the database with keyword and metadata constraints. In this example we allow to look for:

- **Filter Terms:** retrieve only passages that contain exactly the given keywords.
- **Year Range:** retrieve only the records which are inside the provided years
- **Neighbors:** This indicates how much to *expand* the search into more datapoints. The idea is to retrieve the *top_k* neighbors of the initially retrieved passages. Ideally this will give related passages that did not mention any of the keywords explicitly.

In [9]:
widget_year_range = widgets.IntRangeSlider(
    description="Year Range: ",
    min=1800,
    max=2020,
    step=1,
    value=(1800, 1984),
    style={"description_width": "initial"},
    layout=widgets.Layout(width="400px"),
)
widget_terms = widgets.Text(
    description="Filter Terms (comma separated)",
    value="duurzaam",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="600px"),
)
widget_neighbors = widgets.IntSlider(
    description="Expand Neighborhood Size: ",
    min=0,
    max=10,
    value=5,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="400px"),
)

### Display the Widgets to choose the parameters

In [10]:
display(widget_terms)
display(widget_year_range)
display(widget_neighbors)

Text(value='duurzaam', description='Filter Terms (comma separated)', layout=Layout(width='600px'), style=TextS…

IntRangeSlider(value=(1800, 1984), description='Year Range: ', layout=Layout(width='400px'), max=2020, min=180…

IntSlider(value=5, description='Expand Neighborhood Size: ', layout=Layout(width='400px'), max=10, style=Slide…

### Execute the Search

No need to move the code manually here. All parameters are grabbed from the widget values

In [11]:
# Unpack values form Widget
year_from, year_to = widget_year_range.value
FILTER_TERMS = [s.strip() for s in widget_terms.value.split(",")]
# Execute Database Query
where_range = {"year_from": year_from, "year_to": year_to}
print(f"Searching terms {FILTER_TERMS} between year {year_from} and {year_to}")
corpus = sum(
    (
        db.get_corpus(
            collection,
            filter_words=FILTER_TERMS,
            where_obj=where_range,
            include_embeddings=True,
            limit=10000,
        )
        for collection in collection_selector.value
    ),
    start=Corpus([]),
)
print(f"Found {len(corpus)} items that match!")

04:44:56 WARNING:Dropping existing embeddings to avoid inconsistent vector spaces.
04:44:56 WARNING:Dropping existing embeddings to avoid inconsistent vector spaces.


Searching terms ['duurzaam'] between year 1800 and 1984
Found 203 items that match!


### Execute the search for neighborhoods

For each `Passage` in the `Corpus` created with the search result, we will find *k* neighbors and add them to the original `Corpus`.

In [12]:
import statistics


def get_corpus_with_neighborhoods(collection, corpus, k_neighbors):
    all_neighbors = []
    all_distances = []
    for p in corpus.passages:
        neighbors = db.query_vector_neighbors(
            collection, vector=p.embedding, k_neighbors=k_neighbors
        )
        for passage, distance in neighbors:
            all_neighbors.append(passage)
            all_distances.append(distance)
    print(f"Total Datapoints in the neighborhoods = {len(all_neighbors)}")
    if all_distances:
        print(
            f"Distance Info: Max = {max(all_distances)} | Min = {min(all_distances)} | Average = {statistics.mean(all_distances)}"
        )
    else:
        print("No distance information found")
        return Corpus()
    # Join original passages + new found neighbors
    all_passages = corpus.passages + all_neighbors
    corpus = Corpus(all_passages)
    corpus.embeddings = db.compress_embeddings(corpus)
    print(corpus.embeddings.shape)
    return corpus


corpus = get_corpus_with_neighborhoods(
    collection, corpus, k_neighbors=widget_neighbors.value
)

Total Datapoints in the neighborhoods = 1015
Distance Info: Max = 0.4963873028755188 | Min = -1.0728836059570312e-06 | Average = 0.19747920752746131
UMAP( verbose=True)
Mon Aug 12 16:44:59 2024 Construct fuzzy simplicial set


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Mon Aug 12 16:44:59 2024 Finding Nearest Neighbors
Mon Aug 12 16:45:00 2024 Finished Nearest Neighbor Search
Mon Aug 12 16:45:01 2024 Construct embedding


/Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings/.venv/lib/python3.11/site-packages/umap/spectral.py:550: UserWarning: Spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Mon Aug 12 16:45:03 2024 Finished embedding
(1218, 2)


## Cluster the Corpus

### Choose Minimum Cluster Size

In [13]:
min_cluster_size_widget = widgets.IntSlider(
    value=10,
    min=5,
    max=len(corpus) // 2,
    step=1,
    description="Minimum Cluster Size:",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
    style={"description_width": "initial"},
)

In [14]:
min_cluster_size_widget

IntSlider(value=10, continuous_update=False, description='Minimum Cluster Size:', max=609, min=5, style=Slider…

### Execute Clustering

In [15]:
# Arguments: min_cluster_size=10, cluster_selection_epsilon=0.1, ...
# See https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html for full list

# e.g. min_samples=10, cluster_selection_epsilon=0.2, cluster_selection_method="leaf"
clusters = corpus.cluster(
    min_cluster_size=min_cluster_size_widget.value, cluster_selection_epsilon=0.1
)
print(
    f"Found {len(clusters)} clusters in the corpus. (min cluster size is {min_cluster_size_widget.value})"
)

if len(clusters) > 100:
    raise ValueError(
        "Seems like you have too many clusters! Try with a bigger value for min_cluster_size to avoid memory issues"
    )

for c in clusters:
    print(len(c.passages), c)

Found 54 clusters in the corpus. (min cluster size is 10)
18 Subcorpus(8, [0, 3, 4, 203, 204, 205, 206, 207, 218, 219])
19 Subcorpus(37, [1, 96, 166, 182, 208, 209, 210, 211, 212, 267])
238 Subcorpus(-1, [2, 8, 13, 14, 24, 27, 28, 32, 38, 48])
26 Subcorpus(4, [5, 15, 17, 106, 228, 229, 230, 231, 232, 278])
19 Subcorpus(38, [6, 37, 144, 233, 234, 235, 236, 237, 388, 389])
30 Subcorpus(0, [7, 30, 138, 149, 202, 238, 239, 240, 241, 242])
17 Subcorpus(12, [9, 51, 199, 248, 249, 250, 251, 252, 458, 459])
15 Subcorpus(48, [10, 80, 173, 253, 603, 604, 605, 606, 607, 629])
12 Subcorpus(31, [11, 26, 258, 259, 260, 261, 262, 333, 334, 335])
10 Subcorpus(14, [12, 21, 263, 264, 265, 266, 308, 309, 310, 311])
18 Subcorpus(11, [16, 158, 184, 283, 284, 285, 286, 287, 993, 994])
28 Subcorpus(27, [18, 99, 123, 150, 176, 293, 294, 295, 698, 699])
16 Subcorpus(9, [19, 186, 190, 298, 299, 300, 1133, 1134, 1135, 1136])
28 Subcorpus(33, [20, 171, 178, 181, 303, 304, 305, 306, 307, 343])
25 Subcorpus(39, [22

### Load Stopwords to avoid including them in the Cluster Labels

In [16]:
stopwords_file = Path("stopwords-filter-nl.txt")

with open(stopwords_file.absolute(), "rt") as f:
    stopwords = set(f.read().splitlines())

stopwords.update(
    {
        "wij",
        "we",
        "moeten",
        "heer",
        "mevrouw",
        "minister",
        "voorzitter",
        "gaat",
        "wel",
        "den",
    }
)

### Save Cluster Passages for inspection

In [17]:
import os

if not os.path.exists("clusters"):
    os.makedirs("clusters")

selected_metadata = ["year"]

all_clusters_records, df_cluster_labels, df_cluster_meta = [], [], []
for cluster in clusters:
    cluster.set_topic_label(exclude_words=frozenset(stopwords | set(FILTER_TERMS)), n=5)
    df = cluster.to_dataframe()
    centroid = cluster.centroid()
    label = cluster.label
    cluster_size = len(cluster.passages)
    # Compute Cluster Stats as a Subcorpus
    top_words = " ".join(
        cluster.top_words(exclude_words=frozenset(stopwords | set(FILTER_TERMS)), n=10)
    )
    all_clusters_records.append((f"{label}\t{cluster_size}\t{centroid}\t{top_words}\n"))
    df_cluster_labels.append(cluster.label)
    df_cluster_meta.append(df[selected_metadata])
    # Save the Cluster Passages in a File
    file_prefix = f"cluster_{year_from}_{year_to}_{cluster.label.replace('; ', '_')}"
    df.to_csv(f"clusters/{file_prefix}.tsv", sep="\t", index=False)
    print(cluster)

/Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Subcorpus('algemene; beschouwingen; betreffende; energievoorziening; kernenergie', [0, 3, 4, 203, 204, 205, 206, 207, 218, 219])
Subcorpus('defensieapparaat; navo; nucleaire; oorlog; vrede', [1, 96, 166, 182, 208, 209, 210, 211, 212, 267])
Subcorpus('Outliers', [2, 8, 13, 14, 24, 27, 28, 32, 38, 48])
Subcorpus('armoede; eene; mijne; volksklassen; welke', [5, 15, 17, 106, 228, 229, 230, 231, 232, 278])
Subcorpus('mijne; oorlog; stem; wetten; zoude', [6, 37, 144, 233, 234, 235, 236, 237, 388, 389])
Subcorpus('begrooting; benoodigde; geene; uitgaven; zoo', [7, 30, 138, 149, 202, 238, 239, 240, 241, 242])
Subcorpus('beleid; indonesië; motie; orde; regeering', [9, 51, 199, 248, 249, 250, 251, 252, 458, 459])
Subcorpus('congestieheffing; daling; financiering; toekomst; voordeel', [10, 80, 173, 253, 603, 604, 605, 606, 607, 629])
Subcorpus('couranten; daarvan; eeredienst; pamphletten; tijdschriften', [11, 26, 258, 259, 260, 261, 262, 333, 334, 335])
Subcorpus('bereid; communistische; maatrege

In [18]:
with open(f"clusters/clusters_all_{year_from}_{year_to}.csv", "wt") as f:
    f.write("Label\tSize\tCentroid\tTopWords\n")
    for rec in all_clusters_records:
        f.write(rec)

### Plot Cluster Content Distributions

In [19]:
import logging

import matplotlib.pyplot as plt
from ipywidgets import interact

logging.getLogger("matplotlib").setLevel(logging.WARNING)


def plot_cluster_distribution(cluster_index, column_name):
    df = df_cluster_meta[cluster_index]
    cluster_name = df_cluster_labels[cluster_index]

    plt.figure(figsize=(10, 6))
    plt.hist(df[column_name], edgecolor="black")
    plt.xlabel(f"{column_name}", fontsize=14)
    plt.ylabel("Frequency", fontsize=14)
    plt.title(f"{cluster_name}", fontsize=16)
    plt.show()


cluster_selector = widgets.Dropdown(
    options=[(lbl, i) for i, lbl in enumerate(df_cluster_labels)],
    description="Choose a Cluster:",
    disabled=False,
    style={"description_width": "initial"},
)

variable_selector = widgets.Dropdown(
    options=selected_metadata,
    description="Choose a Column to Plot:",
    disabled=False,
    style={"description_width": "initial"},
)

interact(
    plot_cluster_distribution,
    cluster_index=cluster_selector,
    column_name=variable_selector,
)

/Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings/.venv/lib/python3.11/site-packages/ipywidgets/widgets/interaction.py:43: DeprecationWarning: `ipykernel.pylab.backend_inline` is deprecated, directly use `matplotlib_inline.backend_inline`
  from ipykernel.pylab.backend_inline import flush_figures


interactive(children=(Dropdown(description='Choose a Cluster:', options=(('algemene; beschouwingen; betreffend…

<function __main__.plot_cluster_distribution(cluster_index, column_name)>

## Visualize Embeddings (All Clusters)

In [20]:
import os

from bokeh.io import output_notebook
from bokeh.plotting import show

from tempo_embeddings.visualization.bokeh import BokehInteractiveVisualizer

output_notebook()

meta_fields = corpus.metadata_fields()
meta_fields = ["year", "date", "issue", "provenance"]

visualizer = BokehInteractiveVisualizer(
    *clusters, metadata_fields=meta_fields, width=2000, height=1000
)

os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "*"

show(visualizer.create_document)

Loading BokehJS ...

04:45:14 INFO:Starting Bokeh server version 3.3.2 (running on Tornado 6.3.2)
04:45:14 INFO:User authentication hooks NOT provided (default user enabled)


In [21]:
corpus.passages[0].hover_data()

{'text': "1^ %ua(< T*j?^^;< / ^<< ^ ^/ t 18 percent bijdragen^aae?d**H*7*energiebehoeftej ^--------? ^^< aan olie^ De ±HHHHÈHBBt overproductie^ jvan het ogenblik zal dan ook niet duurzaam zijn^' vh*ws*è#%a de deskundigen^ de toestand ^WE zoals die bijvoorbeeld in Tex^s bestaat( waar maar tien dagen per maand wordt geproduceerd) zal binnen afzienbare tijd veranderen! - -/ Over de concurrentie van de kernenergie- kernenergie maken de deelnemers aan Ret wereld-oliecongres zich Bron: Regels: 650. !.",
 'sentence_index': 8.0,
 'year': '1959',
 'date': '02-06-1959',
 'day': '2',
 'provenance': 'ANP_1959.csv.gz',
 'month': '6',
 'filename': 'anp_1959_06_02_41_ocr.xml',
 'issue': '41'}